In [1]:
import numpy as np
import pandas as pd
import math
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits.mplot3d import axes3d
import seaborn as sns
#%matplotlib notebook
sns.set()
import os
import time
import random
import pygraphviz as pgv
import cvxpy as cp
import scipy as scipy
import statistics as stat
from scipy.spatial import KDTree
import random
import json
import math
# Generate points in circle
pi = math.pi
def PointsInCircum(r,n):
    l=[((math.cos(2*pi/n*x)*r)+600,(math.sin(2*pi/n*x)*r)+600) for x in range(0,n)]
    return l

In [2]:
##time series on edges
N=20
##time series on node coordinates

# M_nd=2*N #multiplied by 2 because of x and y coordinated for each node
# X_nd=np.empty((M_nd,0)) #this will the system series for nodes
# MUs_nd=np.empty((M_nd,0))
# Y_nd=np.empty((M_nd,0)) #this will the observed series for nodes
# Y_nd=np.append(Y_nd,np.zeros((M_nd,1)),axis=1)

# l = np.array(PointsInCircum(400, M_nd))
# m=0
# L=np.empty((M_nd,1))
# for i in range(0,M_nd,2):
#     L[i]=l[m,0]
#     L[i+1]=l[m,1]
#     m=m+1
    
# X_nd=np.append(X_nd,L,axis=1) #initializing node positions at 0th time point


# t=100 #time points

# # assign parameter values to your liking
# # C=0.95*np.eye(M)+0.01*np.random.normal(0, 1,(M,M))
# C_nd=np.eye(M_nd)
# # D_nd=1.1*np.eye(M_nd)+0.2*np.random.normal(0, 1,(M_nd,M_nd))
# random.seed(89)
# D_nd=np.eye(M_nd)+3*np.random.normal(0, 0.01,(M_nd,M_nd))
# # D_nd=np.eye(M_nd)
# Gam_nd=1*np.eye(M_nd)
# sigsq1_nd=100
# Q_nd=sigsq1_nd*np.eye(M_nd)
# # Q=0.2*np.random.normal(0, 1,(M,M))+np.eye(M)
# sigsq2_nd=10 #10 #0.1
# Lam_nd=sigsq2_nd*np.eye(M_nd)

# random.seed(65)

# #generate series
# MUs_nd=np.append(MUs_nd,np.array([np.random.multivariate_normal(np.zeros(M_nd), Q_nd)]).T,axis=1)
# Xprev_nd=X_nd.copy()
# mu_nd=MUs_nd.copy()
# for i in range(t):
#     Xnew_nd=np.dot(C_nd,Xprev_nd)+mu_nd #+np.dot(Gam,mu)
#     X_nd=np.append(X_nd,Xnew_nd,axis=1)
#     random.seed(i+86)
#     w_nd=np.array([np.random.multivariate_normal(np.zeros(M_nd), Lam_nd)]).T
#     Ynew_nd=np.dot(D_nd,Xnew_nd)+w_nd
#     Y_nd=np.append(Y_nd,Ynew_nd,axis=1)
#     Xprev_nd=Xnew_nd.copy()
#     random.seed(i+59)
#     mu_nd=np.array([np.random.multivariate_normal(np.zeros(M_nd), Q_nd)]).T
#     MUs_nd=np.append(MUs_nd,mu_nd,axis=1)
    
# print(X_nd.shape)
# print(Y_nd.shape)

M=int(N*(N-1)/2) #number of unique edges, since the graph is symmetric
S = 0.3
X=np.empty((M,0)) #this will the system series for edges
MUs=np.empty((M,0))
Y=np.empty((M,0)) #this will the observed series for edges
Y=np.append(Y,np.zeros((M,1)),axis=1)
random.seed(42)
X=np.append(X,np.random.rand(M,1),axis=1) #initializing edge weights at 0th time point
# ind = np.random.choice(M,int(0.3*M),replace=False)
# X[ind]=0
t=1000 #time points
# assign parameter values to your liking
C=0.95*np.eye(M)
# upper_ind = np.triu_indices(M)
# C_r_ind = np.random.choice(upper_ind[0],int(S*len(upper_ind[0])),replace=False)
# C_c_ind = np.random.choice(upper_ind[1],int(S*len(upper_ind[1])),replace=False)
# ind = (C_r_ind,C_c_ind)
# C[ind]=np.random.normal(0,0.005,C[ind].shape[0])
# C[C==0].shape[0]/(M*M)
#C=0.95*np.eye(M)
#D=2*np.eye(M)+0.1*np.random.normal(0, 1,(M,M))

random.seed(32)
D=np.eye(M)+0.001*np.random.normal(0, 1,(M,M))
# D_ind = np.random.choice(M*M,int(0.6*M*M),replace=False)
# D = D.flatten()
# D[D_ind]=0
# D= D.reshape((M,M))
# D[range(M),range(M)]=1

#D=np.eye(M)
Gam=1*np.eye(M)
sigsq1=0.01 #1
Q=sigsq1*np.eye(M)
#Q=0.2*np.random.normal(0, 1,(M,M))+np.eye(M)
sigsq2=0.1 #0.01
Lam=sigsq2*np.eye(M)
random.seed(54)

In [3]:
#generate series
MUs=np.append(MUs,np.array([np.random.multivariate_normal(np.zeros(M), Q)]).T,axis=1)
Xprev=X.copy()
mu=MUs.copy()
for i in range(t): 
    Xnew=np.dot(C,Xprev)+mu #+np.dot(Gam,mu)
    # Xnew[Xnew<1e-5]=0
    X=np.append(X,Xnew,axis=1)
    random.seed(i+23)
    w=np.array([np.random.multivariate_normal(np.zeros(M), Lam)]).T
    Ynew=np.dot(D,Xnew)+w
    Y=np.append(Y,Ynew,axis=1)
    Xprev=Xnew.copy()
    random.seed(i+76)
    mu=np.array([np.random.multivariate_normal(np.zeros(M), Q)]).T
    MUs=np.append(MUs,mu,axis=1)

In [4]:
Lam_E = np.eye(M)
Q_E = np.eye(M)    
C_E = np.eye(M)
D_E = np.eye(M)

In [5]:
    P = np.empty((M,M,t+1))
    Xk =np.empty((M,2))
    Xp = np.empty((M,t+1))
    Xk[:,1]=Y[:,1]
    Xk[:,0]=X[:,0]
    P[:,:,0]=np.eye(M)*10
    PE = np.empty((M,M,t))
    C_s = scipy.sparse.csc_matrix(C_E)
    D_s = scipy.sparse.csc_matrix(D_E)
    dift1 = []
    dift2 = []
    dift3 = []
    dift4 = []

    for i in range(1,t+1):
        st1 = time.time()
        Xp[:,i] = C_s@Xk[:,i]
        PE[:,:,i-1]= C_s@P[:,:,i-1]@C_s.T+Q_E
        SK =D_s@PE[:,:,i-1]@D_s.T+Lam_E
        st2 = time.time()

        if i%1 == 0 or i ==1:
            sk_inv = np.linalg.inv(SK)
        # else:
        #     L = np.matmul(np.matmul(U.T,SK),U)
        #     Linv = np.diag(1/np.diag(L))
        
        et1 = time.time()
        
        # sk_inv = np.matmul(np.matmul(U,Linv),U.T)
        
        Kk = PE[:,:,i-1]@D_s.T@sk_inv
        
        P[:,:,i]=np.matmul((np.eye(M)-Kk@D_s),PE[:,:,i-1])
        
        Xknew = Xp[:,i].reshape(M,1)+np.matmul(Kk,(Y[:,i].reshape(M,1)-D_s@Xp[:,i].reshape(M,1)))
        zt = time.time()
        Xk = np.append(Xk,Xknew,axis=1)
        dift1 = np.append(dift1,st2-st1)
        dift2 = np.append(dift2,et1-st2)
        dift3 = np.append(dift3,zt-et1)

In [6]:
    X_s = np.empty((M,t+1))
    X_s[:,-1]=Xk[:,-1]
    P_s = np.empty((M,M,t))
    P_s[:,:,t-1]=P[:,:,t-1]
    for i in range(t-1,0,-1):
        Ht = P[:,:,i-1]@C_E@np.linalg.pinv(PE[:,:,i-1])
        X_s[:,i] = Xk[:,i]+Ht@(X_s[:,i+1]-Xp[:,i])
        P_s[:,:,i-1] = P[:,:,i-1]+Ht@(P_s[:,:,i]-PE[:,:,i-1])@Ht.T
    print('Xs-X',np.linalg.norm(X_s-X))

Xs-X 79.9742793106152


In [7]:
from pykalman import KalmanFilter
kf = KalmanFilter(transition_matrices = C_E, observation_matrices =D_E,transition_covariance=Q_E,observation_covariance=Lam_E,initial_state_mean = Y[:,1].T, initial_state_covariance=P[:,:,0], em_vars=[
      'transition_matrices', 'observation_matrices',
      'transition_covariance', 'observation_covariance',
    ])

In [8]:
smoothed_state_means=kf.smooth(Y[:,1:].T)[0]
_s_state = smoothed_state_means.T


In [9]:
_s_state[1,:]

array([ 7.66376958e-01,  9.28978039e-01,  8.39031461e-01,  8.91353822e-01,
        6.17642976e-01,  4.17885891e-01,  5.48659008e-01,  4.62472010e-01,
        5.12232043e-01,  4.85567741e-01,  5.15818772e-01,  4.42469045e-01,
        5.54862669e-01,  6.28268892e-01,  6.52452539e-01,  5.61851548e-01,
        5.60530480e-01,  5.26088140e-01,  6.37019566e-01,  5.78827817e-01,
        6.47618313e-01,  3.86534035e-01,  4.53617101e-01,  4.29651645e-01,
        6.68498447e-01,  9.91879425e-01,  1.09111733e+00,  1.27448152e+00,
        1.25331573e+00,  1.19985451e+00,  1.26136306e+00,  1.34732109e+00,
        1.34099211e+00,  1.01621471e+00,  6.37297536e-01,  7.04788351e-01,
        6.23642027e-01,  5.63422550e-01,  5.11182661e-01,  5.90409718e-01,
        5.36120037e-01,  3.85921041e-01,  4.79694663e-01,  6.39771773e-01,
        5.84662638e-01,  3.49165357e-01,  4.71908360e-01,  4.00306578e-01,
        1.32872112e-01,  3.98793458e-01,  6.45107720e-01,  3.64417821e-01,
        2.12188438e-01,  

In [15]:
    X_s = np.empty((M,t+1))
    X_s[:,-1]=Xk[:,-1]
    P_s = np.empty((M,M,t))
    P_s[:,:,-1]=P[:,:,-1]
    for i in range(t-1,0,-1):
        Ht = P[:,:,i]@C_E@np.linalg.pinv(PE[:,:,i])
        X_s[:,i] = Xk[:,i]+Ht@(X_s[:,i+1]-Xp[:,i])
        P_s[:,:,i-1] = P[:,:,i]+Ht@(P_s[:,:,i]-PE[:,:,i])@Ht.T
    print('Xs-X',np.linalg.norm(X_s-X))

Xs-X 80.00945564604167


In [17]:
X_s[1,:]

array([ 0.        ,  0.67783224,  0.74249528, ..., -0.02435603,
       -0.1403388 , -0.12847865])

In [19]:
(filtered_state_estimates,f_cov) = kf.filter(Y[:,1:].T)

In [25]:
P[:,:,-3]

array([[0.61803399, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.61803399, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.61803399, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.61803399, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.61803399,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.61803399]])

In [27]:
f_cov.T[:,:,-1]

array([[0.61803399, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.61803399, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.61803399, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.61803399, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.61803399,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.61803399]])

In [29]:
P.shape

(190, 190, 1001)

In [30]:
f_cov.shape

(1000, 190, 190)

In [35]:
X_s[1,-3:]-_s_state[1,-3:]

array([0.2385766 , 0.30909466, 0.        ])

In [36]:
i = t-1

In [37]:
Ht = P[:,:,i]@C_E@np.linalg.pinv(PE[:,:,i])

In [38]:
X_s[:,i] = Xk[:,i]+Ht@(X_s[:,i+1]-Xp[:,i])

In [39]:
X_s[1,-2]

-0.1403388026860052

In [42]:
Ht = P[:,:,i]@C_E@np.linalg.inv(PE[:,:,i-1])
X_s[:,i] = Xk[:,i]+Ht@(X_s[:,i+1]-Xp[:,i])

In [44]:
X_s[1,i]

-0.1403388026860052

In [45]:
Ht = P[:,:,i]@C_E@np.linalg.pinv(PE[:,:,i])
X_s[:,i] = Xk[:,i]+Ht@(X_s[:,i+1]-Xp[:,i+1])

In [46]:
X_s[1,i]

0.050692200538389354

In [47]:
Ht = P[:,:,i]@C_E@np.linalg.pinv(PE[:,:,i])
X_s[:,i] = Xk[:,i]+Ht@(X_s[:,i+1]-Xp[:,i-1])

In [48]:
X_s[1,i]

-0.21481990056864772

In [49]:
Xp[1,-10:]

array([ 0.05857389, -0.54819775, -0.16137139,  0.10624474, -0.07297158,
        0.08205317, -0.17304235,  0.04732526, -0.14766878, -0.64779444])

In [50]:
_s_state.shape

(190, 1000)

In [60]:
filtered_state_estimates.shape

(1000, 190)

In [66]:
Ht = P[:,:,i]@C_E@np.linalg.pinv(PE[:,:,-1])
X_s[:,i] = Xk[:,i]+Ht@(X_s[:,i+1]-Xp[:,-1])

In [67]:
X_s[1,i]

0.050692200538389354

In [68]:
smoothed_state_means[-10:,1]

array([-0.3698438 , -0.08126104,  0.04836022, -0.04529889, -0.00052353,
       -0.13413543, -0.07118271, -0.26293263, -0.44943346, -0.12847865])

In [69]:
X_s[1,-2:]

array([ 0.0506922 , -0.12847865])

In [70]:
Ht

array([[0.38196601, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.38196601, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.38196601, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.38196601, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.38196601,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.38196601]])

In [71]:
X_s.shape

(190, 1001)

In [72]:
t-1

999

In [76]:
Ht = P[:,:,-2]@C_E@np.linalg.pinv(PE[:,:,-1])
X_s[:,-2] = Xk[:,-2]+Ht@(X_s[:,-1]-Xp[:,-1])

In [78]:
X_s[1,-2]

-0.4494334588080664

In [81]:
    X_s = np.empty((M,t+1))
    X_s[:,-1]=Xk[:,-1]
    P_s = np.empty((M,M,t))
    P_s[:,:,-1]=P[:,:,-1]
    for i in reversed(range(1,t)):
        print(i)
        Ht = P[:,:,i]@C_E@np.linalg.pinv(PE[:,:,i])
        X_s[:,i] = Xk[:,i]+Ht@(X_s[:,i+1]-Xp[:,i])
        P_s[:,:,i-1] = P[:,:,i]+Ht@(P_s[:,:,i]-PE[:,:,i])@Ht.T
    print('Xs-X',np.linalg.norm(X_s-X))

999
998
997
996
995
994
993
992
991
990
989
988
987
986
985
984
983
982
981
980
979
978
977
976
975
974
973
972
971
970
969
968
967
966
965
964
963
962
961
960
959
958
957
956
955
954
953
952
951
950
949
948
947
946
945
944
943
942
941
940
939
938
937
936
935
934
933
932
931
930
929
928
927
926
925
924
923
922
921
920
919
918
917
916
915
914
913
912
911
910
909
908
907
906
905
904
903
902
901
900
899
898
897
896
895
894
893
892
891
890
889
888
887
886
885
884
883
882
881
880
879
878
877
876
875
874
873
872
871
870
869
868
867
866
865
864
863
862
861
860
859
858
857
856
855
854
853
852
851
850
849
848
847
846
845
844
843
842
841
840
839
838
837
836
835
834
833
832
831
830
829
828
827
826
825
824
823
822
821
820
819
818
817
816
815
814
813
812
811
810
809
808
807
806
805
804
803
802
801
800
799
798
797
796
795
794
793
792
791
790
789
788
787
786
785
784
783
782
781
780
779
778
777
776
775
774
773
772
771
770
769
768
767
766
765
764
763
762
761
760
759
758
757
756
755
754
753
752
751
750


In [82]:
X_s[1,-2]

-0.1403388026860052

In [83]:
Ht = P[:,:,-2]@C_E@np.linalg.pinv(PE[:,:,-1])
X_s[:,-2] = Xk[:,-2]+Ht@(X_s[:,-1]-Xp[:,-1])

In [84]:
X_s[1,-2]

-0.4494334588080664

In [91]:
    X_s = np.empty((M,t+1))
    X_s[:,-1]=Xk[:,-1]
    P_s = np.empty((M,M,t))
    P_s[:,:,-1]=P[:,:,-1]
    for i in reversed(range(1,t)):
        print(i)
        Ht = P[:,:,i]@C_E@np.linalg.pinv(PE[:,:,i])
        X_s[:,i] = Xk[:,i+1]+Ht@(X_s[:,i+1]-Xp[:,i+1])
        P_s[:,:,i-1] = P[:,:,i]+Ht@(P_s[:,:,i]-PE[:,:,i])@Ht.T

999
998
997
996
995
994
993
992
991
990
989
988
987
986
985
984
983
982
981
980
979
978
977
976
975
974
973
972
971
970
969
968
967
966
965
964
963
962
961
960
959
958
957
956
955
954
953
952
951
950
949
948
947
946
945
944
943
942
941
940
939
938
937
936
935
934
933
932
931
930
929
928
927
926
925
924
923
922
921
920
919
918
917
916
915
914
913
912
911
910
909
908
907
906
905
904
903
902
901
900
899
898
897
896
895
894
893
892
891
890
889
888
887
886
885
884
883
882
881
880
879
878
877
876
875
874
873
872
871
870
869
868
867
866
865
864
863
862
861
860
859
858
857
856
855
854
853
852
851
850
849
848
847
846
845
844
843
842
841
840
839
838
837
836
835
834
833
832
831
830
829
828
827
826
825
824
823
822
821
820
819
818
817
816
815
814
813
812
811
810
809
808
807
806
805
804
803
802
801
800
799
798
797
796
795
794
793
792
791
790
789
788
787
786
785
784
783
782
781
780
779
778
777
776
775
774
773
772
771
770
769
768
767
766
765
764
763
762
761
760
759
758
757
756
755
754
753
752
751
750


In [92]:
X_s[1,-2]

-0.4494334588080664

In [93]:
print('Xs-X',np.linalg.norm(X_s-X))

Xs-X 74.5166607793607


In [97]:
X_s[1,1:]-_s_state[1,:]

array([ 7.86340170e-04,  3.00355218e-04,  1.14725485e-04,  4.38212358e-05,
        1.67382226e-05,  6.39343214e-06,  2.44207377e-06,  9.32789178e-07,
        3.56293761e-07,  1.36092107e-07,  5.19825593e-08,  1.98555708e-08,
        7.58415308e-09,  2.89688873e-09,  1.10651299e-09,  4.22650248e-10,
        1.61437974e-10,  6.16637852e-11,  2.35533815e-11,  8.99669228e-12,
        3.43636231e-12,  1.31250566e-12,  5.01321207e-13,  1.91457961e-13,
        7.31636973e-14,  2.79776202e-14,  1.08801856e-14,  4.21884749e-15,
        1.55431223e-15,  6.66133815e-16,  4.44089210e-16,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  1.11022302e-16,  1.11022302e-16,
        1.11022302e-16,  1.11022302e-16,  1.11022302e-16,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00, -5.55111512e-17,  5.55111512e-17,  5.55111512e-17,
        2.77555756e-17,  0.00000000e+00,  0.00000000e+00, -5.55111512e-17,
        0.00000000e+00, -

In [99]:
np.linalg.norm(X_s[1,1:]-_s_state[1,:])

0.0008508550901738017

In [100]:
np.linalg.norm(X_s[:,1:]-_s_state[:,:])

0.011963691771060981

In [101]:
P.shape

(190, 190, 1001)

In [106]:
Xk.shape

(190, 1002)

In [107]:
    X_s = np.empty((M,t+1))
    X_s[:,-1]=Xk[:,-1]
    P_s = np.empty((M,M,t))
    P_s[:,:,-1]=P[:,:,-1]
    for i in reversed(range(0,t)):
        print(i)
        Ht = P[:,:,i]@C_E@np.linalg.pinv(PE[:,:,i])
        X_s[:,i] = Xk[:,i+1]+Ht@(X_s[:,i+1]-Xp[:,i+1])
        P_s[:,:,i-1] = P[:,:,i]+Ht@(P_s[:,:,i]-PE[:,:,i])@Ht.T
    print('Xs-X',np.linalg.norm(X_s-X))

999
998
997
996
995
994
993
992
991
990
989
988
987
986
985
984
983
982
981
980
979
978
977
976
975
974
973
972
971
970
969
968
967
966
965
964
963
962
961
960
959
958
957
956
955
954
953
952
951
950
949
948
947
946
945
944
943
942
941
940
939
938
937
936
935
934
933
932
931
930
929
928
927
926
925
924
923
922
921
920
919
918
917
916
915
914
913
912
911
910
909
908
907
906
905
904
903
902
901
900
899
898
897
896
895
894
893
892
891
890
889
888
887
886
885
884
883
882
881
880
879
878
877
876
875
874
873
872
871
870
869
868
867
866
865
864
863
862
861
860
859
858
857
856
855
854
853
852
851
850
849
848
847
846
845
844
843
842
841
840
839
838
837
836
835
834
833
832
831
830
829
828
827
826
825
824
823
822
821
820
819
818
817
816
815
814
813
812
811
810
809
808
807
806
805
804
803
802
801
800
799
798
797
796
795
794
793
792
791
790
789
788
787
786
785
784
783
782
781
780
779
778
777
776
775
774
773
772
771
770
769
768
767
766
765
764
763
762
761
760
759
758
757
756
755
754
753
752
751
750


In [110]:
# GXs=[] #system
# t = 100
# for tm in range(t):
#     Xmat = np.zeros((N, N))
#     Xmat[np.triu_indices(N, 1)] = X[:,tm].copy()
#     Xmat= Xmat+ Xmat.T - np.diag(Xmat.diagonal())

    
#     GX = nx.from_numpy_matrix(Xmat)
#     m=0
#     for i in range(N):
#         GX.nodes[i]['pos']=[X_nd[m,tm],X_nd[m+1,tm]]
#         m=m+2
#     null_nodes=[]
#     for nd in GX.nodes:
#         if GX.nodes[nd]['pos']==[0,0]:
#             null_nodes.append(nd)
#     GX.remove_nodes_from(null_nodes)
#     GX=nx.relabel.convert_node_labels_to_integers(GX,first_label=0)
#     labels={nn: str(nn) for nn in range(len(GX))}
#     nx.set_node_attributes(GX, labels, 'label')  
#     GXs.append(GX)


# for ts in range(5):
#     plt.figure(figsize=(5,5))
#     GX=GXs[ts].copy()
#     pos=nx.get_node_attributes(GX, 'pos')
#     labels=nx.get_node_attributes(GX, 'label')
#     weight = nx.get_edge_attributes(GX, 'weight')
#     wts=list(weight.values())
#     min_wts=min(wts)
#     wts=[abs(min_wts)+0.001+ele for ele in wts]
#     wts = [i * 0.5 for i in wts]
#     nx.draw(GX,pos=pos,width=wts,labels=labels, font_size=13, node_color='lightcoral',edge_color='lightcoral',font_color='white')
def Para_est(Xk,Y,M,t):
    C_E_U = np.sum(np.array([Xk[:,i].reshape(M,1)@\
        Xk[:,i-1].reshape(M,1).T for i in range(1,t+1)]),axis=0)
    C_E_D = np.sum(np.array([Xk[:,i-1].reshape(M,1)@\
        Xk[:,i-1].reshape(M,1).T for i in range(1,t+1)]),axis=0)
    C_E = C_E_U@np.linalg.pinv(C_E_D)
    D_E_U = np.sum(np.array([Y[:,i].reshape(M,1)@\
        Xk[:,i].reshape(M,1).T for i in range(1,t+1)]),axis=0)
    D_E_D = np.sum(np.array([Xk[:,i].reshape(M,1)@Xk[:,i].reshape(M,1).T for i in range(1,t+1)]),axis=0)
    D_E = D_E_U@np.linalg.inv(D_E_D)
    Lam_E = 1/(t-1)*np.sum(np.array([(Y[:,i].reshape(M,1)-\
        D_E@Xk[:,i].reshape(M,1))@(Y[:,i].reshape(M,1)-\
            D_E@Xk[:,i].reshape(M,1)).T for i in range(1,t+1)]),axis=0)
    Q_E =  1/(t-1)*np.sum(np.array([(Xk[:,i].reshape(M,1)-\
        C_E@Xk[:,i-1].reshape(M,1))@(Xk[:,i].reshape(M,1)-\
            C_E@Xk[:,i-1].reshape(M,1)).T for i in range(1,t+1)]),axis=0)
    return C_E, D_E, Lam_E, Q_E

In [111]:
C_E, D_E, Lam_E, Q_E = Para_est(X_s,Y,M,t)

In [114]:
Lam_E = np.eye(M)
Q_E = np.eye(M)    
C_E = np.eye(M)
D_E = np.eye(M)
_final = np.empty((M,t+1))

while True:
    _final_temp = _final
    P = np.empty((M,M,t+1))
    Xk =np.empty((M,2))
    Xp = np.empty((M,t+1))
    Xk[:,1]=Y[:,1]
    Xk[:,0]=X[:,0]
    P[:,:,0]=np.eye(M)*10
    PE = np.empty((M,M,t))
    C_s = scipy.sparse.csc_matrix(C_E)
    D_s = scipy.sparse.csc_matrix(D_E)
    dift1 = []
    dift2 = []
    dift3 = []
    dift4 = []

    for i in range(1,t+1):
        st1 = time.time()
        Xp[:,i] = C_s@Xk[:,i]
        PE[:,:,i-1]= C_s@P[:,:,i-1]@C_s.T+Q_E
        SK =D_s@PE[:,:,i-1]@D_s.T+Lam_E
        st2 = time.time()

        if i%1 == 0 or i ==1:
            sk_inv = np.linalg.inv(SK)
        # else:
        #     L = np.matmul(np.matmul(U.T,SK),U)
        #     Linv = np.diag(1/np.diag(L))
        
        et1 = time.time()
        
        # sk_inv = np.matmul(np.matmul(U,Linv),U.T)
        
        Kk = PE[:,:,i-1]@D_s.T@sk_inv
        
        P[:,:,i]=np.matmul((np.eye(M)-Kk@D_s),PE[:,:,i-1])
        
        Xknew = Xp[:,i].reshape(M,1)+np.matmul(Kk,(Y[:,i].reshape(M,1)-D_s@Xp[:,i].reshape(M,1)))
        zt = time.time()
        Xk = np.append(Xk,Xknew,axis=1)
        dift1 = np.append(dift1,st2-st1)
        dift2 = np.append(dift2,et1-st2)
        dift3 = np.append(dift3,zt-et1)

    print('Xk-X',np.linalg.norm(Xk[:,1:t+1]-X[:,1:]))

    X_s = np.empty((M,t+1))
    X_s[:,-1]=Xk[:,-1]
    P_s = np.empty((M,M,t))
    P_s[:,:,-1]=P[:,:,-1]
    for i in reversed(range(0,t)):
        
        Ht = P[:,:,i]@C_E@np.linalg.pinv(PE[:,:,i])
        X_s[:,i] = Xk[:,i+1]+Ht@(X_s[:,i+1]-Xp[:,i+1])
        P_s[:,:,i-1] = P[:,:,i]+Ht@(P_s[:,:,i]-PE[:,:,i])@Ht.T
    print('Xs-X',np.linalg.norm(X_s-X))
    _final = X_s
    print(np.linalg.norm(_final_temp - _final)<1e-5)
    if(np.linalg.norm(_final_temp - _final)<1e-5):
        break
    C_E, D_E, Lam_E, Q_E = Para_est(X_s,Y,M,t)
    # C_E[C_E<1e-5]=0
    # D_E[D_E<1e-5]=0
    # Lam_E[Lam_E<1e-5]=0
    # Q_E[Q_E<1e-5]=0
    # print(np.linalg.norm(C_E - C) )
    # print(np.linalg.norm(Q_E - Q) )
    # print(np.linalg.norm(D_E - D) )
    # print(np.linalg.norm(Lam_E - Lam) )

print(np.mean(dift1)+np.mean(dift2)+np.mean(dift3))

Xk-X 103.75655067017556
Xs-X 74.15289582749227
False
Xk-X 80.67630095981579
Xs-X 69.4911052327055
False
Xk-X 78.16010379446178


<ipython-input-114-50cc74e6ac3b>:62: RuntimeWarning: overflow encountered in matmul
  P_s[:,:,i-1] = P[:,:,i]+Ht@(P_s[:,:,i]-PE[:,:,i])@Ht.T
<ipython-input-114-50cc74e6ac3b>:62: RuntimeWarning: invalid value encountered in matmul
  P_s[:,:,i-1] = P[:,:,i]+Ht@(P_s[:,:,i]-PE[:,:,i])@Ht.T


Xs-X inf
False


<ipython-input-110-a3fcdc47df02>:39: RuntimeWarning: overflow encountered in matmul
  C_E_U = np.sum(np.array([Xk[:,i].reshape(M,1)@\
c:\Users\tangm\anaconda3\envs\env_pytorch\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
<ipython-input-110-a3fcdc47df02>:41: RuntimeWarning: overflow encountered in matmul
  C_E_D = np.sum(np.array([Xk[:,i-1].reshape(M,1)@\


LinAlgError: SVD did not converge

In [119]:
P[P>5]

array([10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
       10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
       10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
       10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
       10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
       10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
       10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
       10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
       10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
       10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
       10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
       10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
       10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
       10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10

In [124]:
P_s[:,:,-1]

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [125]:
P[:,:,-1]

array([[ 4.29581469e-03,  2.42638829e-04,  7.64410447e-04, ...,
        -2.26291664e-04,  7.79403771e-05,  5.63899992e-04],
       [ 2.42638829e-04,  4.95431557e-03,  3.28552832e-04, ...,
        -3.91617809e-04,  3.57032743e-04,  2.48881849e-04],
       [ 7.64410447e-04,  3.28552832e-04,  5.25703788e-03, ...,
        -2.58189781e-04,  2.79694892e-04, -4.63620690e-04],
       ...,
       [-2.26291664e-04, -3.91617809e-04, -2.58189781e-04, ...,
         4.57983954e-03,  1.46542810e-04,  4.96009063e-05],
       [ 7.79403771e-05,  3.57032743e-04,  2.79694892e-04, ...,
         1.46542810e-04,  3.59606274e-03, -1.70486068e-04],
       [ 5.63899992e-04,  2.48881849e-04, -4.63620690e-04, ...,
         4.96009063e-05, -1.70486068e-04,  5.07449192e-03]])

In [127]:
 np.seterr(all = "raise")

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [128]:
    X_s = np.empty((M,t+1))
    X_s[:,-1]=Xk[:,-1]
    P_s = np.empty((M,M,t))
    P_s[:,:,-1]=P[:,:,-1]
    for i in reversed(range(0,t)):
        try:
            Ht = P[:,:,i]@C_E@np.linalg.pinv(PE[:,:,i])
            X_s[:,i] = Xk[:,i+1]+Ht@(X_s[:,i+1]-Xp[:,i+1])
            P_s[:,:,i-1] = P[:,:,i]+Ht@(P_s[:,:,i]-PE[:,:,i])@Ht.T
        except FloatingPointError:
            print(i)
            break

484


In [129]:
P_s[:,:,484]

array([[-6.46833220e+306,  7.40721644e+306,  1.14467633e+305, ...,
         4.38851089e+306, -6.83231619e+306,  1.40117184e+307],
       [ 7.40721644e+306, -8.48238058e+306, -1.31082714e+305, ...,
        -5.02550720e+306,  7.82403304e+306, -1.60455320e+307],
       [ 1.14467633e+305, -1.31082714e+305, -2.02569050e+303, ...,
        -7.76618208e+304,  1.20908921e+305, -2.47960092e+305],
       ...,
       [ 4.38851089e+306, -5.02550720e+306, -7.76618208e+304, ...,
        -2.97743333e+306,  4.63545982e+306, -9.50640397e+306],
       [-6.83231619e+306,  7.82403304e+306,  1.20908921e+305, ...,
         4.63545982e+306, -7.21678218e+306,  1.48001815e+307],
       [ 1.40117184e+307, -1.60455320e+307, -2.47960092e+305, ...,
        -9.50640397e+306,  1.48001815e+307, -3.03522217e+307]])

In [131]:
PE[:,:,484]

array([[ 1.49453023e-02, -4.48440829e-04,  1.59630230e-03, ...,
        -3.47496127e-04,  3.17300837e-04,  1.20240620e-03],
       [-4.48440829e-04,  1.67923738e-02, -1.13753665e-03, ...,
        -3.12641894e-05, -6.89616230e-04,  3.11291282e-04],
       [ 1.59630230e-03, -1.13753665e-03,  1.79132927e-02, ...,
        -9.71670193e-04,  6.03133709e-04, -8.79722884e-04],
       ...,
       [-3.47496127e-04, -3.12641894e-05, -9.71670193e-04, ...,
         1.62156551e-02, -2.10325613e-04, -5.20912783e-05],
       [ 3.17300837e-04, -6.89616230e-04,  6.03133709e-04, ...,
        -2.10325613e-04,  1.47982143e-02, -5.96032067e-04],
       [ 1.20240620e-03,  3.11291282e-04, -8.79722884e-04, ...,
        -5.20912783e-05, -5.96032067e-04,  1.57608775e-02]])

In [132]:
P[:,:,i]@C_E@np.linalg.pinv(PE[:,:,i])

array([[ 0.46667709, -0.11528685,  0.05986972, ..., -0.03352305,
         0.05392684, -0.0792101 ],
       [ 0.02360811,  0.40368549,  0.25435036, ..., -0.17590032,
         0.07414925, -0.05989927],
       [-0.0365693 ,  0.0076702 ,  0.26089983, ...,  0.16485423,
         0.00452956, -0.00776604],
       ...,
       [-0.13240533,  0.04138336, -0.08586522, ...,  0.16106218,
         0.12933399, -0.0128574 ],
       [-0.04965594,  0.17167284, -0.02341488, ..., -0.04796277,
         0.28158175,  0.04799016],
       [ 0.13689446, -0.17117728, -0.02004006, ...,  0.10632524,
         0.08824321,  0.44138341]])

In [134]:
P_s[:,:,485]

array([[-1.61272728e+306,  1.84681610e+306,  2.85398260e+304, ...,
         1.09417251e+306, -1.70347818e+306,  3.49349413e+306],
       [ 1.84681610e+306, -2.11488312e+306, -3.26824075e+304, ...,
        -1.25299263e+306,  1.95073957e+306, -4.00057795e+306],
       [ 2.85398260e+304, -3.26824075e+304, -5.05058529e+302, ...,
        -1.93631579e+304,  3.01458103e+304, -6.18230471e+304],
       ...,
       [ 1.09417251e+306, -1.25299263e+306, -1.93631579e+304, ...,
        -7.42353333e+305,  1.15574344e+306, -2.37019939e+306],
       [-1.70347818e+306,  1.95073957e+306,  3.01458103e+304, ...,
         1.15574344e+306, -1.79933577e+306,  3.69007896e+306],
       [ 3.49349413e+306, -4.00057795e+306, -6.18230471e+304, ...,
        -2.37019939e+306,  3.69007896e+306, -7.56761628e+306]])

In [136]:
P_s[:,:,998]

array([[-0.00080474,  0.00086671,  0.0004122 , ...,  0.00129921,
         0.00047975, -0.00084014],
       [ 0.00086671, -0.0032785 ,  0.00093989, ..., -0.00050145,
         0.00018996,  0.0014979 ],
       [ 0.0004122 ,  0.00093989, -0.00248659, ...,  0.0016754 ,
        -0.00011645,  0.00046601],
       ...,
       [ 0.00129921, -0.00050145,  0.0016754 , ..., -0.00094715,
        -0.0002959 ,  0.0007104 ],
       [ 0.00047975,  0.00018996, -0.00011645, ..., -0.0002959 ,
        -0.00016206,  0.001921  ],
       [-0.00084014,  0.0014979 ,  0.00046601, ...,  0.0007104 ,
         0.001921  , -0.00351079]])

In [137]:
P_s[:,:,997]

array([[-0.00892563,  0.0035684 , -0.00021126, ...,  0.00540628,
         0.00123935, -0.0021967 ],
       [ 0.0035684 , -0.02152015,  0.00461495, ..., -0.00209953,
         0.00079909,  0.00302619],
       [-0.00021126,  0.00461495, -0.01594819, ...,  0.00642093,
        -0.0005397 ,  0.00091391],
       ...,
       [ 0.00540628, -0.00209953,  0.00642093, ..., -0.01084578,
        -0.00045682,  0.00229649],
       [ 0.00123935,  0.00079909, -0.0005397 , ..., -0.00045682,
        -0.00645859,  0.00666725],
       [-0.0021967 ,  0.00302619,  0.00091391, ...,  0.00229649,
         0.00666725, -0.0192047 ]])

In [140]:
P_s[:,:,995]

array([[-0.0727427 ,  0.04631414, -0.01036215, ...,  0.04939712,
        -0.00904402,  0.03223297],
       [ 0.04631414, -0.21171257,  0.0557103 , ..., -0.04429187,
         0.03212129, -0.01123184],
       [-0.01036215,  0.0557103 , -0.11259867, ...,  0.05813707,
        -0.00485438, -0.00416375],
       ...,
       [ 0.04939712, -0.04429187,  0.05813707, ..., -0.09835521,
         0.01887385, -0.00698859],
       [-0.00904402,  0.03212129, -0.00485438, ...,  0.01887385,
        -0.06579434,  0.06633181],
       [ 0.03223297, -0.01123184, -0.00416375, ..., -0.00698859,
         0.06633181, -0.19238161]])

In [141]:
X_s[:,i+1]-Xp[:,i+1]

array([ 5.21074333e+153, -5.96708742e+153, -9.22125579e+151,
       -3.41171609e+153,  2.19937956e+153, -1.96197108e+154,
       -2.75098745e+153, -1.43360926e+153,  6.15407001e+152,
       -9.79066322e+152,  4.02726261e+153,  1.27148944e+154,
        8.10013997e+153,  6.68865083e+153,  7.57909228e+153,
        2.38904469e+153,  1.65949228e+154, -6.19999238e+153,
        5.19820436e+153, -5.87724632e+153,  6.46231004e+152,
        2.73448306e+153,  7.52732517e+153,  3.47647114e+153,
       -3.29027247e+153,  4.05286258e+153,  1.26678791e+154,
        2.98495349e+152,  5.10412971e+153,  7.22731570e+153,
       -6.96695505e+153, -4.01868300e+153, -2.61294441e+152,
       -3.14201734e+153, -1.75438777e+153,  4.25420591e+153,
       -3.10701556e+153, -3.10368666e+153, -1.80837239e+154,
       -7.44603972e+153,  5.16110102e+152, -1.52074045e+154,
        8.10843364e+153,  5.80554345e+153, -1.30915046e+154,
        4.56220808e+153,  8.74208187e+153, -4.66670046e+153,
       -8.52327598e+152,

In [145]:
 X_s[:,-1]=Xk[:,-1]

In [152]:
# GXs=[] #system
# t = 100
# for tm in range(t):
#     Xmat = np.zeros((N, N))
#     Xmat[np.triu_indices(N, 1)] = X[:,tm].copy()
#     Xmat= Xmat+ Xmat.T - np.diag(Xmat.diagonal())

    
#     GX = nx.from_numpy_matrix(Xmat)
#     m=0
#     for i in range(N):
#         GX.nodes[i]['pos']=[X_nd[m,tm],X_nd[m+1,tm]]
#         m=m+2
#     null_nodes=[]
#     for nd in GX.nodes:
#         if GX.nodes[nd]['pos']==[0,0]:
#             null_nodes.append(nd)
#     GX.remove_nodes_from(null_nodes)
#     GX=nx.relabel.convert_node_labels_to_integers(GX,first_label=0)
#     labels={nn: str(nn) for nn in range(len(GX))}
#     nx.set_node_attributes(GX, labels, 'label')  
#     GXs.append(GX)


# for ts in range(5):
#     plt.figure(figsize=(5,5))
#     GX=GXs[ts].copy()
#     pos=nx.get_node_attributes(GX, 'pos')
#     labels=nx.get_node_attributes(GX, 'label')
#     weight = nx.get_edge_attributes(GX, 'weight')
#     wts=list(weight.values())
#     min_wts=min(wts)
#     wts=[abs(min_wts)+0.001+ele for ele in wts]
#     wts = [i * 0.5 for i in wts]
#     nx.draw(GX,pos=pos,width=wts,labels=labels, font_size=13, node_color='lightcoral',edge_color='lightcoral',font_color='white')
def Para_est(Xk,Y,M,t):
    C_E_U = np.sum(np.array([Xk[:,i].reshape(M,1)@\
        Xk[:,i-1].reshape(M,1).T for i in range(2,t+1)]),axis=0)
    C_E_D = np.sum(np.array([Xk[:,i-1].reshape(M,1)@\
        Xk[:,i-1].reshape(M,1).T for i in range(2,t+1)]),axis=0)
    C_E = C_E_U@np.linalg.pinv(C_E_D)
    D_E_U = np.sum(np.array([Y[:,i].reshape(M,1)@\
        Xk[:,i].reshape(M,1).T for i in range(2,t+1)]),axis=0)
    D_E_D = np.sum(np.array([Xk[:,i].reshape(M,1)@\
        Xk[:,i].reshape(M,1).T for i in range(2,t+1)]),axis=0)
    D_E = D_E_U@np.linalg.inv(D_E_D)
    Lam_E = 1/(t-1)*np.sum(np.array([(Y[:,i].reshape(M,1)-\
        D_E@Xk[:,i].reshape(M,1))@(Y[:,i].reshape(M,1)-\
            D_E@Xk[:,i].reshape(M,1)).T for i in range(2,t+1)]),axis=0)
    Q_E =  1/(t-1)*np.sum(np.array([(Xk[:,i].reshape(M,1)-\
        C_E@Xk[:,i-1].reshape(M,1))@(Xk[:,i].reshape(M,1)-\
            C_E@Xk[:,i-1].reshape(M,1)).T for i in range(2,t+1)]),axis=0)
    return C_E, D_E, Lam_E, Q_E

In [158]:
# GXs=[] #system
# t = 100
# for tm in range(t):
#     Xmat = np.zeros((N, N))
#     Xmat[np.triu_indices(N, 1)] = X[:,tm].copy()
#     Xmat= Xmat+ Xmat.T - np.diag(Xmat.diagonal())

    
#     GX = nx.from_numpy_matrix(Xmat)
#     m=0
#     for i in range(N):
#         GX.nodes[i]['pos']=[X_nd[m,tm],X_nd[m+1,tm]]
#         m=m+2
#     null_nodes=[]
#     for nd in GX.nodes:
#         if GX.nodes[nd]['pos']==[0,0]:
#             null_nodes.append(nd)
#     GX.remove_nodes_from(null_nodes)
#     GX=nx.relabel.convert_node_labels_to_integers(GX,first_label=0)
#     labels={nn: str(nn) for nn in range(len(GX))}
#     nx.set_node_attributes(GX, labels, 'label')  
#     GXs.append(GX)


# for ts in range(5):
#     plt.figure(figsize=(5,5))
#     GX=GXs[ts].copy()
#     pos=nx.get_node_attributes(GX, 'pos')
#     labels=nx.get_node_attributes(GX, 'label')
#     weight = nx.get_edge_attributes(GX, 'weight')
#     wts=list(weight.values())
#     min_wts=min(wts)
#     wts=[abs(min_wts)+0.001+ele for ele in wts]
#     wts = [i * 0.5 for i in wts]
#     nx.draw(GX,pos=pos,width=wts,labels=labels, font_size=13, node_color='lightcoral',edge_color='lightcoral',font_color='white')
def Para_est(Xk,Y,M,t):
    C_E_U = np.sum(np.array([Xk[:,i].reshape(M,1)@\
        Xk[:,i-1].reshape(M,1).T for i in range(1,t+1)]),axis=0)
    C_E_D = np.sum(np.array([Xk[:,i-1].reshape(M,1)@\
        Xk[:,i-1].reshape(M,1).T for i in range(1,t+1)]),axis=0)
    C_E = C_E_U@np.linalg.pinv(C_E_D)
    D_E_U = np.sum(np.array([Y[:,i].reshape(M,1)@\
        Xk[:,i].reshape(M,1).T for i in range(1,t+1)]),axis=0)
    D_E_D = np.sum(np.array([Xk[:,i].reshape(M,1)@\
        Xk[:,i].reshape(M,1).T for i in range(1,t+1)]),axis=0)
    D_E = D_E_U@np.linalg.inv(D_E_D)
    Lam_E = 1/(t-1)*np.sum(np.array([(Y[:,i].reshape(M,1)-\
        D_E@Xk[:,i].reshape(M,1))@(Y[:,i].reshape(M,1)-\
            D_E@Xk[:,i].reshape(M,1)).T for i in range(1,t+1)]),axis=0)
    Q_E =  1/(t-1)*np.sum(np.array([(Xk[:,i].reshape(M,1)-\
        C_E@Xk[:,i-1].reshape(M,1))@(Xk[:,i].reshape(M,1)-\
            C_E@Xk[:,i-1].reshape(M,1)).T for i in range(1,t+1)]),axis=0)
    return C_E, D_E, Lam_E, Q_E

In [167]:
Lam_E = np.eye(M)
Q_E = np.eye(M)    
C_E = np.eye(M)
D_E = np.eye(M)
_final = np.empty((M,t+1))

while True:
    _final_temp = _final
    P = np.empty((M,M,t+1))
    Xk =np.empty((M,2))
    Xp = np.empty((M,t+1))
    Xk[:,1]=Y[:,1]
    Xk[:,0]=X[:,0]
    P[:,:,0]=np.eye(M)*10
    PE = np.empty((M,M,t))
    C_s = scipy.sparse.csc_matrix(C_E)
    D_s = scipy.sparse.csc_matrix(D_E)
    dift1 = []
    dift2 = []
    dift3 = []
    dift4 = []

    for i in range(1,t+1):
        st1 = time.time()
        Xp[:,i] = C_s@Xk[:,i]
        PE[:,:,i-1]= C_s@P[:,:,i-1]@C_s.T+Q_E
        SK =D_s@PE[:,:,i-1]@D_s.T+Lam_E
        st2 = time.time()

        if i%1 == 0 or i ==1:
            sk_inv = np.linalg.inv(SK)
        # else:
        #     L = np.matmul(np.matmul(U.T,SK),U)
        #     Linv = np.diag(1/np.diag(L))
        
        et1 = time.time()
        
        # sk_inv = np.matmul(np.matmul(U,Linv),U.T)
        
        Kk = PE[:,:,i-1]@D_s.T@sk_inv
        
        P[:,:,i]=np.matmul((np.eye(M)-Kk@D_s),PE[:,:,i-1])
        
        Xknew = Xp[:,i].reshape(M,1)+np.matmul(Kk,(Y[:,i].reshape(M,1)-D_s@Xp[:,i].reshape(M,1)))
        zt = time.time()
        Xk = np.append(Xk,Xknew,axis=1)
        dift1 = np.append(dift1,st2-st1)
        dift2 = np.append(dift2,et1-st2)
        dift3 = np.append(dift3,zt-et1)

    print('Xk-X',np.linalg.norm(Xk[:,1:t+1]-X[:,1:]))
    flag = False
    X_s = np.empty((M,t+1))
    X_s[:,-1]=Xk[:,-1]
    X_s[:,0]=Xk[:,1]
    P_s = np.empty((M,M,t))
    P_s[:,:,-1]=P[:,:,-1]
    for i in reversed(range(1,t)):
        try:
            Ht = P[:,:,i]@C_E@np.linalg.pinv(PE[:,:,i])
            X_s[:,i] = Xk[:,i+1]+Ht@(X_s[:,i+1]-Xp[:,i+1])
            P_s[:,:,i-1] = P[:,:,i]+Ht@(P_s[:,:,i]-PE[:,:,i])@Ht.T
            if np.linalg.norm(X_s[:,i])>1e1:
                print(i)
                flag = True
                break
                
        except FloatingPointError:
            print(i)
            break

        
    if flag == True:
        break
    print('Xs-X',np.linalg.norm(X_s-X))
    _final = X_s
    print(np.linalg.norm(_final_temp - _final)<1e-5)
    if(np.linalg.norm(_final_temp - _final)<1e-5):
        break
    C_E, D_E, Lam_E, Q_E = Para_est(X_s,Y,M,t)
    # C_E[C_E<1e-5]=0
    # D_E[D_E<1e-5]=0
    # Lam_E[Lam_E<1e-5]=0
    # Q_E[Q_E<1e-5]=0
    # print(np.linalg.norm(C_E - C) )
    # print(np.linalg.norm(Q_E - Q) )
    # print(np.linalg.norm(D_E - D) )
    # print(np.linalg.norm(Lam_E - Lam) )

print(np.mean(dift1)+np.mean(dift2)+np.mean(dift3))

Xk-X 103.75655067017556
Xs-X 74.21421202447172
False
Xk-X 80.6728505776018
Xs-X 69.18713496895359
False
Xk-X 78.14982894423396
995
0.028313701391220095


In [173]:
X_s[1,1:].shape

(1000,)

In [175]:
Lam_E = np.eye(M)
Q_E = np.eye(M)    
C_E = np.eye(M)
D_E = np.eye(M)


In [176]:
    P = np.empty((M,M,t+1))
    Xk =np.empty((M,2))
    Xp = np.empty((M,t+1))
    Xk[:,1]=Y[:,1]
    P[:,:,0]=np.eye(M)*10
    PE = np.empty((M,M,t))
    C_s = scipy.sparse.csc_matrix(C_E)
    D_s = scipy.sparse.csc_matrix(D_E)
    dift1 = []
    dift2 = []
    dift3 = []
    dift4 = []

    for i in range(1,t+1):
        st1 = time.time()
        Xp[:,i] = C_s@Xk[:,i]
        PE[:,:,i-1]= C_s@P[:,:,i-1]@C_s.T+Q_E
        SK =D_s@PE[:,:,i-1]@D_s.T+Lam_E
        st2 = time.time()

        if i%1 == 0 or i ==1:
            sk_inv = np.linalg.inv(SK)
        # else:
        #     L = np.matmul(np.matmul(U.T,SK),U)
        #     Linv = np.diag(1/np.diag(L))
        
        et1 = time.time()
        
        # sk_inv = np.matmul(np.matmul(U,Linv),U.T)
        
        Kk = PE[:,:,i-1]@D_s.T@sk_inv
        
        P[:,:,i]=np.matmul((np.eye(M)-Kk@D_s),PE[:,:,i-1])
        
        Xknew = Xp[:,i].reshape(M,1)+np.matmul(Kk,(Y[:,i].reshape(M,1)-D_s@Xp[:,i].reshape(M,1)))
        zt = time.time()
        Xk = np.append(Xk,Xknew,axis=1)
        dift1 = np.append(dift1,st2-st1)
        dift2 = np.append(dift2,et1-st2)
        dift3 = np.append(dift3,zt-et1)

    print('Xk-X',np.linalg.norm(Xk[:,1:t+1]-X[:,1:]))
    flag = False
    X_s = np.empty((M,t+1))
    X_s[:,-1]=Xk[:,-1]
    P_s = np.empty((M,M,t))
    P_s[:,:,-1]=P[:,:,-1]
    for i in reversed(range(0,t)):
        try:
            Ht = P[:,:,i]@C_E@np.linalg.pinv(PE[:,:,i])
            X_s[:,i] = Xk[:,i+1]+Ht@(X_s[:,i+1]-Xp[:,i+1])
            P_s[:,:,i-1] = P[:,:,i]+Ht@(P_s[:,:,i]-PE[:,:,i])@Ht.T
            if np.linalg.norm(X_s[:,i])>1e1:
                print(i)
                flag = True
                break
                
        except FloatingPointError:
            print(i)
            break

Xk-X 103.75655067017556


In [184]:
np.linalg.norm(Y[:,0]-X[:,1])

7.814089702797292

In [188]:
_s_state[1,0]

0.7663769576275771

In [189]:
X_s[1,1]

0.717428391233488

In [192]:
X_s[1,0]

1.3718484936471047

In [193]:
Xk[:,1:].shape

(190, 198)

In [195]:
Lam_E = np.eye(M)
Q_E = np.eye(M)    
C_E = np.eye(M)
D_E = np.eye(M)

In [196]:
initial_guess = Y[:,1]

In [197]:
    P = np.empty((M,M,t+1))
    Xk =np.empty((M,2))
    Xp = np.empty((M,t+1))
    Xk[:,1]=Y[:,1]
    P[:,:,0]=np.eye(M)*10
    PE = np.empty((M,M,t))
    C_s = scipy.sparse.csc_matrix(C_E)
    D_s = scipy.sparse.csc_matrix(D_E)
    dift1 = []
    dift2 = []
    dift3 = []
    dift4 = []

    for i in range(1,t+1):
        st1 = time.time()
        Xp[:,i] = C_s@Xk[:,i]
        PE[:,:,i-1]= C_s@P[:,:,i-1]@C_s.T+Q_E
        SK =D_s@PE[:,:,i-1]@D_s.T+Lam_E
        st2 = time.time()

        if i%1 == 0 or i ==1:
            sk_inv = np.linalg.inv(SK)
        # else:
        #     L = np.matmul(np.matmul(U.T,SK),U)
        #     Linv = np.diag(1/np.diag(L))
        
        et1 = time.time()
        
        # sk_inv = np.matmul(np.matmul(U,Linv),U.T)
        
        Kk = PE[:,:,i-1]@D_s.T@sk_inv
        
        P[:,:,i]=np.matmul((np.eye(M)-Kk@D_s),PE[:,:,i-1])
        
        Xknew = Xp[:,i].reshape(M,1)+np.matmul(Kk,(Y[:,i].reshape(M,1)-D_s@Xp[:,i].reshape(M,1)))
        zt = time.time()
        Xk = np.append(Xk,Xknew,axis=1)
        dift1 = np.append(dift1,st2-st1)
        dift2 = np.append(dift2,et1-st2)
        dift3 = np.append(dift3,zt-et1)

In [198]:
Xk[:,1:].shape

(190, 1001)

In [199]:
filtered_state_estimates.T.shape

(190, 1000)

In [200]:
filtered_state_estimates.T[1,:]

array([ 6.18557792e-01,  9.88005481e-01,  8.06406931e-01,  1.06073542e+00,
        7.41122954e-01,  3.37061434e-01,  6.01925716e-01,  4.31718600e-01,
        5.28711490e-01,  4.66871576e-01,  5.61151396e-01,  3.73005965e-01,
        5.09495128e-01,  6.13322576e-01,  7.08447031e-01,  5.62668012e-01,
        5.81817017e-01,  4.57528749e-01,  6.72984046e-01,  5.36312952e-01,
        8.08977271e-01,  3.45074420e-01,  4.68428568e-01,  2.82036203e-01,
        4.68638011e-01,  9.30547027e-01,  9.77792030e-01,  1.28756269e+00,
        1.28635658e+00,  1.16184014e+00,  1.20823808e+00,  1.35123261e+00,
        1.54171559e+00,  1.25039841e+00,  5.95585918e-01,  7.54939536e-01,
        6.60859711e-01,  5.95708577e-01,  4.62217647e-01,  6.23962587e-01,
        6.28948121e-01,  3.27965756e-01,  3.80761568e-01,  6.73831091e-01,
        7.30207961e-01,  2.73306009e-01,  5.16160695e-01,  5.65590168e-01,
       -3.14763189e-02,  2.46562872e-01,  8.18583617e-01,  4.58500754e-01,
        3.03991800e-01,  

In [202]:
Xk[1,1:]

array([ 0.61855779,  0.61855779,  0.98850813, ..., -0.14766878,
       -0.64779444, -0.12847865])

In [204]:
X_s.shape

(190, 1001)

In [205]:
X.shape

(190, 1001)

In [206]:
_s_state[1,-2]

-0.4494334588080664

In [207]:
X_s[1,-3]

-0.2464947416068627

In [209]:
    X_s = np.empty((M,t+1))
    X_s[:,-1]=Xk[:,-1]
    P_s = np.empty((M,M,t))
    P_s[:,:,-1]=P[:,:,-1]
    for i in reversed(range(1,t)):
        try:
            Ht = P[:,:,i]@C_E@np.linalg.pinv(PE[:,:,i])
            X_s[:,i] = Xk[:,i+1]+Ht@(X_s[:,i+1]-Xp[:,i+1])
            P_s[:,:,i-1] = P[:,:,i]+Ht@(P_s[:,:,i]-PE[:,:,i])@Ht.T
            if np.linalg.norm(X_s[:,i])>1e2:
                print(i)
                flag = True
                break
                
        except FloatingPointError:
            print(i)
            break

In [210]:
_s_state[1,0]

0.7663769576275771

In [213]:
X_s[1,1]

0.7671632977977674

In [214]:
_s_state[1,-1]

-0.12847864864309477

In [215]:
X_s[1,-1]

-0.12847864864309477

In [216]:
X_s[1,-2]

-0.4494334588080664

In [217]:
_s_state[1,-2]

-0.4494334588080664

In [218]:
_s_state[1,-3]

-0.26293263131625416

In [219]:
X_s[1,-3]

-0.2629326313162541

In [220]:
_s_state[1,998]

-0.4494334588080664

In [221]:
X_s[1,t-2]

-0.2629326313162541

In [222]:
X_s[1,t-3]

-0.07118270625083926

In [223]:
X_s[1,t-1]

-0.4494334588080664

In [224]:
# GXs=[] #system
# t = 100
# for tm in range(t):
#     Xmat = np.zeros((N, N))
#     Xmat[np.triu_indices(N, 1)] = X[:,tm].copy()
#     Xmat= Xmat+ Xmat.T - np.diag(Xmat.diagonal())

    
#     GX = nx.from_numpy_matrix(Xmat)
#     m=0
#     for i in range(N):
#         GX.nodes[i]['pos']=[X_nd[m,tm],X_nd[m+1,tm]]
#         m=m+2
#     null_nodes=[]
#     for nd in GX.nodes:
#         if GX.nodes[nd]['pos']==[0,0]:
#             null_nodes.append(nd)
#     GX.remove_nodes_from(null_nodes)
#     GX=nx.relabel.convert_node_labels_to_integers(GX,first_label=0)
#     labels={nn: str(nn) for nn in range(len(GX))}
#     nx.set_node_attributes(GX, labels, 'label')  
#     GXs.append(GX)


# for ts in range(5):
#     plt.figure(figsize=(5,5))
#     GX=GXs[ts].copy()
#     pos=nx.get_node_attributes(GX, 'pos')
#     labels=nx.get_node_attributes(GX, 'label')
#     weight = nx.get_edge_attributes(GX, 'weight')
#     wts=list(weight.values())
#     min_wts=min(wts)
#     wts=[abs(min_wts)+0.001+ele for ele in wts]
#     wts = [i * 0.5 for i in wts]
#     nx.draw(GX,pos=pos,width=wts,labels=labels, font_size=13, node_color='lightcoral',edge_color='lightcoral',font_color='white')
def Para_est(Xk,Y,M,t):
    C_E_U = np.sum(np.array([Xk[:,i].reshape(M,1)@\
        Xk[:,i-1].reshape(M,1).T for i in range(2,t+1)]),axis=0)
    C_E_D = np.sum(np.array([Xk[:,i-1].reshape(M,1)@\
        Xk[:,i-1].reshape(M,1).T for i in range(2,t+1)]),axis=0)
    C_E = C_E_U@np.linalg.pinv(C_E_D)
    D_E_U = np.sum(np.array([Y[:,i].reshape(M,1)@\
        Xk[:,i].reshape(M,1).T for i in range(1,t+1)]),axis=0)
    D_E_D = np.sum(np.array([Xk[:,i].reshape(M,1)@\
        Xk[:,i].reshape(M,1).T for i in range(1,t+1)]),axis=0)
    D_E = D_E_U@np.linalg.pinv(D_E_D)
    Lam_E = (1/t)*np.sum(np.array([(Y[:,i].reshape(M,1)-\
        D_E@Xk[:,i].reshape(M,1))@(Y[:,i].reshape(M,1)-\
            D_E@Xk[:,i].reshape(M,1)).T for i in range(1,t)]),axis=0)
    Q_E =  (1/t)*np.sum(np.array([(Xk[:,i].reshape(M,1)-\
        C_E@Xk[:,i-1].reshape(M,1))@(Xk[:,i].reshape(M,1)-\
            C_E@Xk[:,i-1].reshape(M,1)).T for i in range(2,t+1)]),axis=0)
    return C_E, D_E, Lam_E, Q_E

In [225]:
Lam_E = np.eye(M)
Q_E = np.eye(M)    
C_E = np.eye(M)
D_E = np.eye(M)
_final = np.empty((M,t+1))
# initial_guess = Y[:,1]
while True:
    _final_temp = _final
    P = np.empty((M,M,t+1))
    Xk =np.empty((M,2))
    Xp = np.empty((M,t+1))
    Xk[:,1]=Y[:,1]
    P[:,:,0]=np.eye(M)*10
    PE = np.empty((M,M,t))
    C_s = scipy.sparse.csc_matrix(C_E)
    D_s = scipy.sparse.csc_matrix(D_E)
    dift1 = []
    dift2 = []
    dift3 = []
    dift4 = []

    for i in range(1,t+1):
        st1 = time.time()
        Xp[:,i] = C_s@Xk[:,i]
        PE[:,:,i-1]= C_s@P[:,:,i-1]@C_s.T+Q_E
        SK =D_s@PE[:,:,i-1]@D_s.T+Lam_E
        st2 = time.time()

        if i%1 == 0 or i ==1:
            sk_inv = np.linalg.inv(SK)
        # else:
        #     L = np.matmul(np.matmul(U.T,SK),U)
        #     Linv = np.diag(1/np.diag(L))
        
        et1 = time.time()
        
        # sk_inv = np.matmul(np.matmul(U,Linv),U.T)
        
        Kk = PE[:,:,i-1]@D_s.T@sk_inv
        
        P[:,:,i]=np.matmul((np.eye(M)-Kk@D_s),PE[:,:,i-1])
        
        Xknew = Xp[:,i].reshape(M,1)+np.matmul(Kk,(Y[:,i].reshape(M,1)-D_s@Xp[:,i].reshape(M,1)))
        zt = time.time()
        Xk = np.append(Xk,Xknew,axis=1)
        dift1 = np.append(dift1,st2-st1)
        dift2 = np.append(dift2,et1-st2)
        dift3 = np.append(dift3,zt-et1)

    print('Xk-X',np.linalg.norm(Xk[:,1:t+1]-X[:,1:]))
    flag = False
    X_s = np.empty((M,t+1))
    X_s[:,-1]=Xk[:,-1]
    P_s = np.empty((M,M,t))
    P_s[:,:,-1]=P[:,:,-1]
    for i in reversed(range(1,t)):
        try:
            Ht = P[:,:,i]@C_E@np.linalg.pinv(PE[:,:,i])
            X_s[:,i] = Xk[:,i+1]+Ht@(X_s[:,i+1]-Xp[:,i+1])
            P_s[:,:,i-1] = P[:,:,i]+Ht@(P_s[:,:,i]-PE[:,:,i])@Ht.T
            if np.linalg.norm(X_s[:,i])>1e2:
                print(i)
                flag = True
                break
                
        except FloatingPointError:
            print(i)
            break

        
    if flag == True:
        break
    print('Xs-X',np.linalg.norm(X_s-X))

    _final = X_s
    print(np.linalg.norm(_final_temp - _final)<1e-5)
    if(np.linalg.norm(_final_temp - _final)<1e-5):
        break
    C_E, D_E, Lam_E, Q_E = Para_est(X_s,Y,M,t)
    # C_E[C_E<1e-5]=0
    # D_E[D_E<1e-5]=0
    # Lam_E[Lam_E<1e-5]=0
    # Q_E[Q_E<1e-5]=0
    # print(np.linalg.norm(C_E - C) )
    # print(np.linalg.norm(Q_E - Q) )
    # print(np.linalg.norm(D_E - D) )
    # print(np.linalg.norm(Lam_E - Lam) )

print(np.mean(dift1)+np.mean(dift2)+np.mean(dift3))

Xk-X 103.75655067017556
Xs-X 74.5166607793607
False
Xk-X 80.67349903302149
Xs-X 69.51770958544415
False
Xk-X 78.15110999807946
992
0.02819159460067749


In [226]:
##time series on edges
N=20
##time series on node coordinates

# M_nd=2*N #multiplied by 2 because of x and y coordinated for each node
# X_nd=np.empty((M_nd,0)) #this will the system series for nodes
# MUs_nd=np.empty((M_nd,0))
# Y_nd=np.empty((M_nd,0)) #this will the observed series for nodes
# Y_nd=np.append(Y_nd,np.zeros((M_nd,1)),axis=1)

# l = np.array(PointsInCircum(400, M_nd))
# m=0
# L=np.empty((M_nd,1))
# for i in range(0,M_nd,2):
#     L[i]=l[m,0]
#     L[i+1]=l[m,1]
#     m=m+1
    
# X_nd=np.append(X_nd,L,axis=1) #initializing node positions at 0th time point


# t=100 #time points

# # assign parameter values to your liking
# # C=0.95*np.eye(M)+0.01*np.random.normal(0, 1,(M,M))
# C_nd=np.eye(M_nd)
# # D_nd=1.1*np.eye(M_nd)+0.2*np.random.normal(0, 1,(M_nd,M_nd))
# random.seed(89)
# D_nd=np.eye(M_nd)+3*np.random.normal(0, 0.01,(M_nd,M_nd))
# # D_nd=np.eye(M_nd)
# Gam_nd=1*np.eye(M_nd)
# sigsq1_nd=100
# Q_nd=sigsq1_nd*np.eye(M_nd)
# # Q=0.2*np.random.normal(0, 1,(M,M))+np.eye(M)
# sigsq2_nd=10 #10 #0.1
# Lam_nd=sigsq2_nd*np.eye(M_nd)

# random.seed(65)

# #generate series
# MUs_nd=np.append(MUs_nd,np.array([np.random.multivariate_normal(np.zeros(M_nd), Q_nd)]).T,axis=1)
# Xprev_nd=X_nd.copy()
# mu_nd=MUs_nd.copy()
# for i in range(t):
#     Xnew_nd=np.dot(C_nd,Xprev_nd)+mu_nd #+np.dot(Gam,mu)
#     X_nd=np.append(X_nd,Xnew_nd,axis=1)
#     random.seed(i+86)
#     w_nd=np.array([np.random.multivariate_normal(np.zeros(M_nd), Lam_nd)]).T
#     Ynew_nd=np.dot(D_nd,Xnew_nd)+w_nd
#     Y_nd=np.append(Y_nd,Ynew_nd,axis=1)
#     Xprev_nd=Xnew_nd.copy()
#     random.seed(i+59)
#     mu_nd=np.array([np.random.multivariate_normal(np.zeros(M_nd), Q_nd)]).T
#     MUs_nd=np.append(MUs_nd,mu_nd,axis=1)
    
# print(X_nd.shape)
# print(Y_nd.shape)

M=int(N*(N-1)/2) #number of unique edges, since the graph is symmetric
S = 0.3
X=np.empty((M,0)) #this will the system series for edges
MUs=np.empty((M,0))
Y=np.empty((M,0)) #this will the observed series for edges
Y=np.append(Y,np.zeros((M,1)),axis=1)
random.seed(42)
X=np.append(X,np.random.rand(M,1),axis=1) #initializing edge weights at 0th time point
# ind = np.random.choice(M,int(0.3*M),replace=False)
# X[ind]=0
t=1000 #time points
# assign parameter values to your liking
C=0.95*np.eye(M)
upper_ind = np.triu_indices(M)
C_r_ind = np.random.choice(upper_ind[0],int(S*len(upper_ind[0])),replace=False)
C_c_ind = np.random.choice(upper_ind[1],int(S*len(upper_ind[1])),replace=False)
ind = (C_r_ind,C_c_ind)
C[ind]=np.random.normal(0,0.005,C[ind].shape[0])
# C[C==0].shape[0]/(M*M)
#C=0.95*np.eye(M)
#D=2*np.eye(M)+0.1*np.random.normal(0, 1,(M,M))

random.seed(32)
D=np.eye(M)+0.001*np.random.normal(0, 1,(M,M))
# D_ind = np.random.choice(M*M,int(0.6*M*M),replace=False)
# D = D.flatten()
# D[D_ind]=0
# D= D.reshape((M,M))
# D[range(M),range(M)]=1

#D=np.eye(M)
Gam=1*np.eye(M)
sigsq1=0.01 #1
Q=sigsq1*np.eye(M)
#Q=0.2*np.random.normal(0, 1,(M,M))+np.eye(M)
sigsq2=0.1 #0.01
Lam=sigsq2*np.eye(M)
random.seed(54)

In [227]:
#generate series
MUs=np.append(MUs,np.array([np.random.multivariate_normal(np.zeros(M), Q)]).T,axis=1)
Xprev=X.copy()
mu=MUs.copy()
for i in range(t): 
    Xnew=np.dot(C,Xprev)+mu #+np.dot(Gam,mu)
    # Xnew[Xnew<1e-5]=0
    X=np.append(X,Xnew,axis=1)
    random.seed(i+23)
    w=np.array([np.random.multivariate_normal(np.zeros(M), Lam)]).T
    Ynew=np.dot(D,Xnew)+w
    Y=np.append(Y,Ynew,axis=1)
    Xprev=Xnew.copy()
    random.seed(i+76)
    mu=np.array([np.random.multivariate_normal(np.zeros(M), Q)]).T
    MUs=np.append(MUs,mu,axis=1)

In [228]:
import numpy as np
import pandas as pd
import math
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits.mplot3d import axes3d
import seaborn as sns
#%matplotlib notebook
sns.set()
import os
import time
import random
import pygraphviz as pgv
import cvxpy as cp
import scipy as scipy
import statistics as stat
from scipy.spatial import KDTree
import random
import json
import math
# Generate points in circle
pi = math.pi
def PointsInCircum(r,n):
    l=[((math.cos(2*pi/n*x)*r)+600,(math.sin(2*pi/n*x)*r)+600) for x in range(0,n)]
    return l

In [230]:
#generate series
MUs=np.append(MUs,np.array([np.random.multivariate_normal(np.zeros(M), Q)]).T,axis=1)
Xprev=X.copy()
mu=MUs.copy()
for i in range(t): 
    Xnew=np.dot(C,Xprev)+mu #+np.dot(Gam,mu)
    # Xnew[Xnew<1e-5]=0
    X=np.append(X,Xnew,axis=1)
    random.seed(i+23)
    w=np.array([np.random.multivariate_normal(np.zeros(M), Lam)]).T
    Ynew=np.dot(D,Xnew)+w
    Y=np.append(Y,Ynew,axis=1)
    Xprev=Xnew.copy()
    random.seed(i+76)
    mu=np.array([np.random.multivariate_normal(np.zeros(M), Q)]).T
    MUs=np.append(MUs,mu,axis=1)

In [231]:
# GXs=[] #system
# t = 100
# for tm in range(t):
#     Xmat = np.zeros((N, N))
#     Xmat[np.triu_indices(N, 1)] = X[:,tm].copy()
#     Xmat= Xmat+ Xmat.T - np.diag(Xmat.diagonal())

    
#     GX = nx.from_numpy_matrix(Xmat)
#     m=0
#     for i in range(N):
#         GX.nodes[i]['pos']=[X_nd[m,tm],X_nd[m+1,tm]]
#         m=m+2
#     null_nodes=[]
#     for nd in GX.nodes:
#         if GX.nodes[nd]['pos']==[0,0]:
#             null_nodes.append(nd)
#     GX.remove_nodes_from(null_nodes)
#     GX=nx.relabel.convert_node_labels_to_integers(GX,first_label=0)
#     labels={nn: str(nn) for nn in range(len(GX))}
#     nx.set_node_attributes(GX, labels, 'label')  
#     GXs.append(GX)


# for ts in range(5):
#     plt.figure(figsize=(5,5))
#     GX=GXs[ts].copy()
#     pos=nx.get_node_attributes(GX, 'pos')
#     labels=nx.get_node_attributes(GX, 'label')
#     weight = nx.get_edge_attributes(GX, 'weight')
#     wts=list(weight.values())
#     min_wts=min(wts)
#     wts=[abs(min_wts)+0.001+ele for ele in wts]
#     wts = [i * 0.5 for i in wts]
#     nx.draw(GX,pos=pos,width=wts,labels=labels, font_size=13, node_color='lightcoral',edge_color='lightcoral',font_color='white')
def Para_est(Xk,Y,M,t):
    C_E_U = np.sum(np.array([Xk[:,i].reshape(M,1)@\
        Xk[:,i-1].reshape(M,1).T for i in range(2,t+1)]),axis=0)
    C_E_D = np.sum(np.array([Xk[:,i-1].reshape(M,1)@\
        Xk[:,i-1].reshape(M,1).T for i in range(2,t+1)]),axis=0)
    C_E = C_E_U@np.linalg.pinv(C_E_D)
    D_E_U = np.sum(np.array([Y[:,i].reshape(M,1)@\
        Xk[:,i].reshape(M,1).T for i in range(1,t+1)]),axis=0)
    D_E_D = np.sum(np.array([Xk[:,i].reshape(M,1)@\
        Xk[:,i].reshape(M,1).T for i in range(1,t+1)]),axis=0)
    D_E = D_E_U@np.linalg.pinv(D_E_D)
    Lam_E = (1/t)*np.sum(np.array([(Y[:,i].reshape(M,1)-\
        D_E@Xk[:,i].reshape(M,1))@(Y[:,i].reshape(M,1)-\
            D_E@Xk[:,i].reshape(M,1)).T for i in range(1,t)]),axis=0)
    Q_E =  (1/t)*np.sum(np.array([(Xk[:,i].reshape(M,1)-\
        C_E@Xk[:,i-1].reshape(M,1))@(Xk[:,i].reshape(M,1)-\
            C_E@Xk[:,i-1].reshape(M,1)).T for i in range(2,t+1)]),axis=0)
    return C_E, D_E, Lam_E, Q_E

In [233]:
print(np.linalg.norm(Y[:,1:]-X[:,1:]))
Lam_E = np.eye(M)
Q_E = np.eye(M)    
C_E = np.eye(M)
D_E = np.eye(M)
_final = np.empty((M,t+1))
# initial_guess = Y[:,1]
while True:
    _final_temp = _final
    P = np.empty((M,M,t+1))
    Xk =np.empty((M,2))
    Xp = np.empty((M,t+1))
    Xk[:,1]=Y[:,1]
    P[:,:,0]=np.eye(M)*10
    PE = np.empty((M,M,t))
    C_s = scipy.sparse.csc_matrix(C_E)
    D_s = scipy.sparse.csc_matrix(D_E)
    dift1 = []
    dift2 = []
    dift3 = []
    dift4 = []

    for i in range(1,t+1):
        st1 = time.time()
        Xp[:,i] = C_s@Xk[:,i]
        PE[:,:,i-1]= C_s@P[:,:,i-1]@C_s.T+Q_E
        SK =D_s@PE[:,:,i-1]@D_s.T+Lam_E
        st2 = time.time()

        if i%1 == 0 or i ==1:
            sk_inv = np.linalg.inv(SK)
        # else:
        #     L = np.matmul(np.matmul(U.T,SK),U)
        #     Linv = np.diag(1/np.diag(L))
        
        et1 = time.time()
        
        # sk_inv = np.matmul(np.matmul(U,Linv),U.T)
        
        Kk = PE[:,:,i-1]@D_s.T@sk_inv
        
        P[:,:,i]=np.matmul((np.eye(M)-Kk@D_s),PE[:,:,i-1])
        
        Xknew = Xp[:,i].reshape(M,1)+np.matmul(Kk,(Y[:,i].reshape(M,1)-D_s@Xp[:,i].reshape(M,1)))
        zt = time.time()
        Xk = np.append(Xk,Xknew,axis=1)
        dift1 = np.append(dift1,st2-st1)
        dift2 = np.append(dift2,et1-st2)
        dift3 = np.append(dift3,zt-et1)

    print('Xk-X',np.linalg.norm(Xk[:,1:t+1]-X[:,1:]))
    flag = False
    X_s = np.empty((M,t+1))
    X_s[:,-1]=Xk[:,-1]
    P_s = np.empty((M,M,t))
    P_s[:,:,-1]=P[:,:,-1]
    for i in reversed(range(1,t)):
        try:
            Ht = P[:,:,i]@C_E@np.linalg.pinv(PE[:,:,i])
            X_s[:,i] = Xk[:,i+1]+Ht@(X_s[:,i+1]-Xp[:,i+1])
            P_s[:,:,i-1] = P[:,:,i]+Ht@(P_s[:,:,i]-PE[:,:,i])@Ht.T
            if np.linalg.norm(X_s[:,i])>1e2:
                print(i)
                flag = True
                break
                
        except FloatingPointError:
            print(i)
            break

        
    if flag == True:
        break
    print('Xs-X',np.linalg.norm(X_s-X))

    _final = X_s
    print(np.linalg.norm(_final_temp - _final)<1e-5)
    if(np.linalg.norm(_final_temp - _final)<1e-5):
        break
    C_E, D_E, Lam_E, Q_E = Para_est(X_s,Y,M,t)
    # C_E[C_E<1e-5]=0
    # D_E[D_E<1e-5]=0
    # Lam_E[Lam_E<1e-5]=0
    # Q_E[Q_E<1e-5]=0
    # print(np.linalg.norm(C_E - C) )
    # print(np.linalg.norm(Q_E - Q) )
    # print(np.linalg.norm(D_E - D) )
    # print(np.linalg.norm(Lam_E - Lam) )

print(np.mean(dift1)+np.mean(dift2)+np.mean(dift3))

137.93351220689038
Xk-X 103.5973338786798
Xs-X 74.51372953537427
False
Xk-X 80.31002065086427
Xs-X 69.24093806642335
False
Xk-X 77.69586904212909
990
0.02740589141845703


In [234]:
Lam_E = np.eye(M)
Q_E = np.eye(M)    
C_E = np.eye(M)
D_E = np.eye(M)

In [236]:
_final_temp = _final
P = np.empty((M,M,t+1))
Xk =np.empty((M,2))
Xp = np.empty((M,t+1))
Xk[:,1]=Y[:,1]
P[:,:,0]=np.eye(M)*10
PE = np.empty((M,M,t))
C_s = scipy.sparse.csc_matrix(C_E)
D_s = scipy.sparse.csc_matrix(D_E)
dift1 = []
dift2 = []
dift3 = []
dift4 = []

for i in range(1,t+1):
    st1 = time.time()
    Xp[:,i] = C_s@Xk[:,i]
    PE[:,:,i-1]= C_s@P[:,:,i-1]@C_s.T+Q_E
    SK =D_s@PE[:,:,i-1]@D_s.T+Lam_E
    st2 = time.time()

    if i%1 == 0 or i ==1:
        sk_inv = np.linalg.inv(SK)
    # else:
    #     L = np.matmul(np.matmul(U.T,SK),U)
    #     Linv = np.diag(1/np.diag(L))
    
    et1 = time.time()
    
    # sk_inv = np.matmul(np.matmul(U,Linv),U.T)
    
    Kk = PE[:,:,i-1]@D_s.T@sk_inv
    
    P[:,:,i]=np.matmul((np.eye(M)-Kk@D_s),PE[:,:,i-1])
    
    Xknew = Xp[:,i].reshape(M,1)+np.matmul(Kk,(Y[:,i].reshape(M,1)-D_s@Xp[:,i].reshape(M,1)))
    zt = time.time()
    Xk = np.append(Xk,Xknew,axis=1)
    dift1 = np.append(dift1,st2-st1)
    dift2 = np.append(dift2,et1-st2)
    dift3 = np.append(dift3,zt-et1)

print('Xk-X',np.linalg.norm(Xk[:,1:t+1]-X[:,1:]))
flag = False
X_s = np.empty((M,t+1))
X_s[:,-1]=Xk[:,-1]
P_s = np.empty((M,M,t))
P_s[:,:,-1]=P[:,:,-1]
for i in reversed(range(1,t)):
    try:
        Ht = P[:,:,i]@C_E@np.linalg.pinv(PE[:,:,i])
        X_s[:,i] = Xk[:,i+1]+Ht@(X_s[:,i+1]-Xp[:,i+1])
        P_s[:,:,i-1] = P[:,:,i]+Ht@(P_s[:,:,i]-PE[:,:,i])@Ht.T
        if np.linalg.norm(X_s[:,i])>1e2:
            print(i)
            flag = True
            break
            
    except FloatingPointError:
        print(i)
        break

    
if flag == True:
    break
print('Xs-X',np.linalg.norm(X_s-X))

Xk-X 103.5973338786798


SyntaxError: 'break' outside loop (<ipython-input-236-1b21dd6707b5>, line 65)

In [237]:
print('Xs-X',np.linalg.norm(X_s-X))

Xs-X 74.51372953537427


In [238]:
kf = KalmanFilter(transition_matrices = C_E, observation_matrices =D_E,transition_covariance=Q_E,observation_covariance=Lam_E,initial_state_mean = Y[:,1].T, initial_state_covariance=P[:,:,0], em_vars=[
      'transition_matrices', 'observation_matrices',
      'transition_covariance', 'observation_covariance',
    ])

In [239]:
(smoothed_state_means, smoothed_state_covariances)=kf.smooth(Y[:,1:].T)

In [240]:
transition_matrices = C_E
observation_matrices =D_E
transition_covariance=Q_E
observation_covariance=Lam_E

In [241]:
observations = Y[:,1:].T

In [244]:
_,_,test_Lam_E,_ = Para_est(X_s,Y,M,t)

In [246]:
_s_state = smoothed_state_means.T

In [247]:
np.linalg.norm(X_s[:,1:]-_s_state[:,:])

0.01035773255218434

In [255]:
_, n_dim_state = smoothed_state_means.shape
n_timesteps, n_dim_obs = observations.shape
res = np.zeros((n_dim_obs, n_dim_obs))
n_obs = 0
for t in range(n_timesteps):
    if not np.any(np.ma.getmask(observations[t])):
        transition_matrix = D_E
        err = (
            observations[t]
            - np.dot(transition_matrix, smoothed_state_means[t])
            
        )
        res += (
            np.outer(err, err)
            + np.dot(transition_matrix,
                        np.dot(smoothed_state_covariances[t],
                            transition_matrix.T))
        )
        n_obs += 1
res = (1.0 / n_obs) * res

In [269]:
np.dot(P_s[:,:,t],transition_matrix.T).shape

(190, 190)

In [271]:
 np.outer(err, err).shape

(190, 190)

In [273]:
np.dot(transition_matrix,
                            np.dot(P_s[:,:,t],
                                transition_matrix.T)).shape

(190, 190)

In [277]:
res

array([[ 4.35428562e-02, -1.94350430e-03,  5.43231670e-03, ...,
         1.45160920e-03, -1.15929779e-03, -3.76387434e-05],
       [-1.94350430e-03,  4.02039186e-02, -3.84204053e-04, ...,
         2.62786820e-03, -1.67985025e-04, -1.43144550e-03],
       [ 5.43231670e-03, -3.84204053e-04,  3.84652946e-02, ...,
         8.19789536e-04, -1.11050730e-03,  9.81096476e-04],
       ...,
       [ 1.45160920e-03,  2.62786820e-03,  8.19789536e-04, ...,
         4.10743917e-02,  9.09546732e-04, -7.59161520e-04],
       [-1.15929779e-03, -1.67985025e-04, -1.11050730e-03, ...,
         9.09546732e-04,  3.73429591e-02,  6.20560534e-04],
       [-3.76387434e-05, -1.43144550e-03,  9.81096476e-04, ...,
        -7.59161520e-04,  6.20560534e-04,  4.03340446e-02]])

In [282]:
def _em_observation_covariance(X_s,Y,P_s):
    n_dim_state,_ = X_s.shape
    print(n_dim_state)
    n_dim_obs,n_timesteps = Y.shape
    res = np.zeros((n_dim_obs, n_dim_obs))
    n_obs = 0
    for t in range(n_timesteps):
        if not np.any(np.ma.getmask(Y[:,t])):
            transition_matrix = D_E
            err = (
                Y[:,t]
                - np.dot(transition_matrix, X_s[:,t])
                
            )
            res += (
                np.outer(err, err)
                + np.dot(transition_matrix,
                            np.dot(P_s[:,:,t],
                                transition_matrix.T))
            )
            n_obs += 1
    res = (1.0 / n_obs) * res

In [286]:
def _em_observation_covariance(X_s,Y,P_s):
    n_dim_state,_ = X_s.shape
    print(n_dim_state)
    n_dim_obs,n_timesteps = Y.shape
    res = np.zeros((n_dim_obs, n_dim_obs))
    n_obs = 0
    for t in range(n_timesteps):
        if not np.any(np.ma.getmask(Y[:,t])):
            transition_matrix = D_E
            err = (
                Y[:,t]
                - np.dot(transition_matrix, X_s[:,t])
                
            )
            res += (
                np.outer(err, err)
                + np.dot(transition_matrix,
                            np.dot(P_s[:,:,t],
                                transition_matrix.T))
            )
            n_obs += 1
    return  (1.0 / n_obs) * res

In [287]:
test_Lam_E = _em_observation_covariance(X_s[:,1:],Y[:,1:],P_s)

190


In [288]:
test_Lam_E

array([[ 4.91117953e-01, -1.94359268e-03,  5.43235290e-03, ...,
         1.45152711e-03, -1.15932035e-03, -3.76414379e-05],
       [-1.94359268e-03,  4.87779503e-01, -3.84404738e-04, ...,
         2.62821300e-03, -1.67871179e-04, -1.43141378e-03],
       [ 5.43235290e-03, -3.84404738e-04,  4.86040481e-01, ...,
         8.19645302e-04, -1.11055479e-03,  9.81083348e-04],
       ...,
       [ 1.45152711e-03,  2.62821300e-03,  8.19645302e-04, ...,
         4.88649725e-01,  9.09626824e-04, -7.59136149e-04],
       [-1.15932035e-03, -1.67871179e-04, -1.11055479e-03, ...,
         9.09626824e-04,  4.84918088e-01,  6.20568269e-04],
       [-3.76414379e-05, -1.43141378e-03,  9.81083348e-04, ...,
        -7.59136149e-04,  6.20568269e-04,  4.87909149e-01]])

In [289]:
t = 1000

In [290]:
_, n_dim_state = smoothed_state_means.shape
n_timesteps, n_dim_obs = observations.shape
res = np.zeros((n_dim_obs, n_dim_obs))
n_obs = 0
for t in range(n_timesteps):
    if not np.any(np.ma.getmask(observations[t])):
        transition_matrix = D_E
        err = (
            observations[t]
            - np.dot(transition_matrix, smoothed_state_means[t])
            
        )
        res += (
            np.outer(err, err)
            + np.dot(transition_matrix,
                        np.dot(smoothed_state_covariances[t],
                            transition_matrix.T))
        )
        n_obs += 1
res = (1.0 / n_obs) * res

In [291]:
res

array([[ 4.91114333e-01, -1.94350430e-03,  5.43231670e-03, ...,
         1.45160920e-03, -1.15929779e-03, -3.76387434e-05],
       [-1.94350430e-03,  4.87775396e-01, -3.84204053e-04, ...,
         2.62786820e-03, -1.67985025e-04, -1.43144550e-03],
       [ 5.43231670e-03, -3.84204053e-04,  4.86036772e-01, ...,
         8.19789536e-04, -1.11050730e-03,  9.81096476e-04],
       ...,
       [ 1.45160920e-03,  2.62786820e-03,  8.19789536e-04, ...,
         4.88645869e-01,  9.09546732e-04, -7.59161520e-04],
       [-1.15929779e-03, -1.67985025e-04, -1.11050730e-03, ...,
         9.09546732e-04,  4.84914436e-01,  6.20560534e-04],
       [-3.76387434e-05, -1.43144550e-03,  9.81096476e-04, ...,
        -7.59161520e-04,  6.20560534e-04,  4.87905522e-01]])

In [292]:
_, n_dim_state = smoothed_state_means.shape
n_timesteps, n_dim_obs = observations.shape
res1 = np.zeros((n_dim_obs, n_dim_state))
res2 = np.zeros((n_dim_state, n_dim_state))
for t in range(n_timesteps):
    if not np.any(np.ma.getmask(observations[t])):
        res1 += np.outer(observations[t],
                            smoothed_state_means[t])
        res2 += (
            smoothed_state_covariances[t]
            + np.outer(smoothed_state_means[t], smoothed_state_means[t])
        )
print(np.dot(res1, np.linalg.pinv(res2)))

[[ 2.04808313e-01 -9.18672756e-03 -6.50441707e-03 ... -1.02986640e-03
   3.20302268e-03  2.25047483e-02]
 [-9.98526168e-03  2.06640462e-01 -2.70471344e-02 ... -1.10113439e-02
  -1.28041995e-02 -1.84963828e-02]
 [-6.49451564e-03 -2.69085206e-02  2.36752843e-01 ...  4.58714124e-03
  -3.05268048e-04 -4.97433567e-03]
 ...
 [-1.52802567e-03 -1.09317712e-02  5.10536204e-03 ...  1.77211477e-01
   1.99537108e-02  1.04513946e-02]
 [ 3.27849861e-03 -1.31756839e-02 -5.38065343e-04 ...  2.00761185e-02
   1.68124133e-01 -9.23176700e-05]
 [ 2.23057187e-02 -1.83470654e-02 -4.78407736e-03 ...  1.05801016e-02
  -3.45681139e-04  1.81219038e-01]]


In [293]:
_,D_E,_,_ = Para_est(X_s,Y,M,t)

In [294]:
D_E

array([[ 1.63328881e+00,  5.73997615e-02,  4.10631099e-03, ...,
         3.39907123e-02, -2.78572641e-02, -5.85796563e-02],
       [ 5.04183534e-03,  1.56298303e+00,  8.51022540e-02, ...,
         7.74550353e-02,  1.67983693e-02,  4.88114252e-02],
       [ 1.08980224e-03,  6.61537653e-02,  1.49634557e+00, ...,
         9.05048173e-03,  2.74445104e-02,  3.15118010e-02],
       ...,
       [-5.21950640e-04,  6.09021526e-02,  4.32851883e-02, ...,
         1.64117817e+00, -3.21732272e-02, -5.14660765e-02],
       [-4.95009469e-02, -2.34546044e-02,  5.88758145e-03, ...,
        -2.10349894e-02,  1.74514926e+00, -1.85457252e-04],
       [-5.26739939e-02,  3.34677851e-02,  5.32511640e-02, ...,
        -5.85389243e-02, -1.54724692e-02,  1.70169753e+00]])

In [295]:
D

array([[ 1.00057610e+00, -4.47760225e-04, -3.01741122e-04, ...,
        -2.02338747e-03,  9.52982512e-04,  1.24109127e-04],
       [-2.61720459e-06,  9.99528902e-01,  3.81354564e-04, ...,
         1.99515307e-03,  1.83884963e-03, -5.68974589e-04],
       [-5.98474605e-04,  3.43231054e-04,  9.99998638e-01, ...,
        -1.22706000e-04,  5.99343293e-04,  1.32046733e-03],
       ...,
       [-5.90042491e-04,  1.57387275e-04,  2.12884909e-04, ...,
         9.99945140e-01,  1.45892335e-03,  1.19269017e-03],
       [-3.82344106e-04, -4.29974184e-04,  4.76580899e-04, ...,
         5.42452159e-04,  1.00122534e+00,  6.15782970e-04],
       [ 1.72398447e-03, -5.29400951e-04,  1.15051575e-03, ...,
         1.07401627e-03, -4.83190299e-04,  9.99842801e-01]])

In [296]:
D_E = np.eye(M)

In [298]:
D_E_U = np.sum(np.array([Y[:,i].reshape(M,1)@\
        X_s[:,i].reshape(M,1).T for i in range(1,t+1)]),axis=0)

In [299]:
D_E_U

array([[154.80286005,  -1.62381928, -29.40906443, ...,  -0.5526601 ,
          2.28872625,  20.62602251],
       [ -1.64221069, 174.76020254, -45.15733918, ..., -15.66663002,
        -18.60528198, -15.33779255],
       [-29.3943874 , -45.16178038, 215.77607172, ...,  12.03828904,
          5.56020416,  -2.33773237],
       ...,
       [ -0.56896825, -15.64311647,  12.01072393, ..., 125.42600417,
         24.93659672,  10.97616431],
       [  2.25855518, -18.59640847,   5.63707496, ...,  24.89254138,
        122.13889695,  -6.09263796],
       [ 20.61211232, -15.32521577,  -2.32744685, ...,  10.96126631,
         -6.04674642, 124.27846701]])

In [300]:
res1

array([[154.80152127,  -1.6248409 , -29.3991917 , ...,  -0.55474207,
          2.27402782,  20.61795363],
       [ -1.64079566, 174.76493181, -45.1771708 , ..., -15.66120353,
        -18.5769031 , -15.32228286],
       [-29.39683749, -45.16612574, 215.80051412, ...,  12.03229044,
          5.52459213,  -2.35723505],
       ...,
       [ -0.57055709, -15.64285046,  12.01863404, ..., 125.42484019,
         24.92435587,  10.96941651],
       [  2.26876621, -18.57250895,   5.52030128, ...,  24.92265899,
        122.30768934,  -6.00028294],
       [ 20.61484126, -15.31831591,  -2.359975  , ...,  10.9697685 ,
         -5.99983189, 124.30412991]])

In [301]:
D_E_D = np.sum(np.array([X_s[:,i].reshape(M,1)@\
        X_s[:,i].reshape(M,1).T for i in range(1,t+1)]),axis=0)

In [302]:
D_E_D

array([[133.04030111,  -0.90008608, -31.02789764, ...,  -1.37636706,
          2.49221487,  20.39412784],
       [ -0.90008608, 154.05700791, -45.12256006, ..., -16.03835471,
        -18.37617744, -13.97171714],
       [-31.02789764, -45.12256006, 196.45063977, ...,  11.5703825 ,
          5.79838962,  -2.61624286],
       ...,
       [ -1.37636706, -16.03835471,  11.5703825 , ..., 104.5945643 ,
         24.07604972,  10.64152758],
       [  2.49221487, -18.37617744,   5.79838962, ...,  24.07604972,
        101.96032972,  -6.39655714],
       [ 20.39412784, -13.97171714,  -2.61624286, ...,  10.64152758,
         -6.39655714, 102.65021807]])

In [307]:
def _em_observation_matrix(X_s,Y,P_s):
    r"""Apply the EM algorithm to parameter `observation_matrix`

    Maximize expected log likelihood of observations with respect to the
    observation matrix `observation_matrix`.

    .. math::

        C &= ( \sum_{t=0}^{T-1} (z_t - d_t) \mathbb{E}[x_t]^T )
             ( \sum_{t=0}^{T-1} \mathbb{E}[x_t x_t^T] )^-1

    """
    n_dim_state,_ = X_s.shape
    n_dim_obs,n_timesteps = Y.shape
    res1 = np.zeros((n_dim_obs, n_dim_state))
    res2 = np.zeros((n_dim_state, n_dim_state))
    for t in range(n_timesteps):
        if not np.any(np.ma.getmask(Y[:,t])):
            res1 += np.outer(Y[:,t],
                             X_s[:,t])
            res2 += (
                P_s[:,:,t]
                + np.outer(X_s[:,t], X_s[:,t])
            )
    return np.dot(res1, np.linalg.pinv(res2))    

In [308]:
_em_observation_matrix(X_s[:,1:],Y[:,1:],P_s)

array([[ 2.04807820e-01, -9.18826974e-03, -6.50402450e-03, ...,
        -1.03068590e-03,  3.20292081e-03,  2.25044915e-02],
       [-9.98426668e-03,  2.06637393e-01, -2.70464355e-02, ...,
        -1.10121753e-02, -1.28042667e-02, -1.84964750e-02],
       [-6.49464154e-03, -2.69091387e-02,  2.36751688e-01, ...,
         4.58643288e-03, -3.05319992e-04, -4.97445544e-03],
       ...,
       [-1.52645696e-03, -1.09351394e-02,  5.10642471e-03, ...,
         1.77208884e-01,  1.99534875e-02,  1.04510281e-02],
       [ 3.27896222e-03, -1.31757996e-02, -5.37789882e-04, ...,
         2.00764285e-02,  1.68123189e-01, -9.22522256e-05],
       [ 2.23058991e-02, -1.83476927e-02, -4.78385492e-03, ...,
         1.05796618e-02, -3.45737948e-04,  1.81217870e-01]])

In [309]:
(smoothed_state_means, smoothed_state_covariances)=kf.em(Y[:,1:].T,em_vars=['transition_matrices','transition_covariance', 'observation_matrices','observation_covariance'],n_iter=2).smooth(Y[:,1:].T)


In [310]:
_s_state = smoothed_state_means.T
np.linalg.norm(_s_state[:,0:]-X[:,1:])

170.6570855529451

In [316]:
Lam_E = np.eye(M)
Q_E = np.eye(M)    
C_E = 5*np.eye(M)
D_E = 3*np.eye(M)

In [317]:
kf = KalmanFilter(transition_matrices = C_E, observation_matrices =D_E,\
    transition_covariance=Q_E,observation_covariance=Lam_E,\
        initial_state_mean = Y[:,1].T, initial_state_covariance=P[:,:,0], em_vars=[
      'transition_matrices', 'observation_matrices',
      'transition_covariance', 'observation_covariance',
    ])

In [318]:
(smoothed_state_means, smoothed_state_covariances)=kf.smooth(Y[:,1:].T)
_s_state = smoothed_state_means.T
np.linalg.norm(_s_state[:,0:]-X[:,1:])

126.65921586917263

In [319]:
(smoothed_state_means, smoothed_state_covariances)=kf.em(Y[:,1:].T,em_vars=['transition_matrices','transition_covariance', 'observation_matrices','observation_covariance'],n_iter=1).smooth(Y[:,1:].T)
_s_state = smoothed_state_means.T
np.linalg.norm(_s_state[:,0:]-X[:,1:])

119.38923225522475

In [320]:
for i in range(2):
    kf = kf.em(X=Y[:,1:].T, n_iter=1)
_s_state = smoothed_state_means.T
np.linalg.norm(_s_state[:,0:]-X[:,1:])

119.38923225522475

In [321]:
np.linalg.norm(Y[:,1:]-X[:,1:])

137.93351220689038

In [322]:
(smoothed_state_means, smoothed_state_covariances)=kf.smooth(Y[:,1:].T)
_s_state = smoothed_state_means.T
np.linalg.norm(_s_state[:,0:]-X[:,1:])

98.98065588999089